# Problem #10
## Asked by Apple
### description

Implement a job-scheduler which takes in a function `f` and an int `n` and calls f after n milliseconds

# Implementation

**Assumptions**

for the purposes of this challenge i will assume the following conditions:
 * calls are one-off, not recurring
 * functions scheduled at the same time will be run sequentially, in the order first recieved by the scheduler (no parallelism)
 * No use of language specific modules which already implement this functionality -
    * can't use sched module
    * can't use threading.timer

**Checking Approaches**
count_down - every several ms 

In [204]:
from time import time_ns
def time_ms():
    """Gets time as nano secs, then convert to ms"""
    return time_ns() // 1_000_000

In [205]:
from time import time,time_ns,sleep
from queue import PriorityQueue

class Scheduler:
    def __init__(self) -> None:
        # When scheduling a call, set it up to be a call as current-time (in milliseconds) + n-miliseconds, then check periodically. use a queue & append schedueled jobs so that 
        self.scheduled_funcs = PriorityQueue()

    def run(self):
        # Limitation : if a new item is scheduled to occur before the item currently held, it will still execute after the held item.
        if not self.scheduled_funcs.empty():
            item = self.scheduled_funcs.get()
            wait_duration = (item[0] - time_ms()) / 1000 # scheduled time
            sleep(wait_duration)
            item[1]()

    def schedule(self,n_ms:int,f:callable,*args):
        """Schedule f to be called after n miliseconds"""
        def bundle_func():
            """Closure: return callable with args"""
            return f(*args)

        scheduled_time = time_ms() + n_ms
        self.scheduled_funcs.put(
            # Priority, data
            (scheduled_time,bundle_func)
        )


In [206]:
def hello(s:str):
    print(f'Hello {s}')

In [207]:
from time import time
def measured_hello(s:str):
    global calls
    print(f'Hello {s}')
    calls.append(
        (time_ms(),s)
        )
    

In [214]:

calls = []
s = Scheduler()
s.schedule(100,
        #    hello,
        measured_hello,
           'World'
           )
s.schedule(150,
        #    hello,
        measured_hello,
           'There'
           )
s.schedule(10,
        #    hello,
        measured_hello,
           'Mike'
           )

start_time = time_ms()
# Correct Order:  Mike -> World -> There
while True:
    s.run()
#     if s.scheduled_funcs.empty():
#         break




Hello Mike
Hello World
Hello There


KeyboardInterrupt: 

In [216]:
for called_time,call_str in calls:
    difference = called_time - start_time
    print(f'called {call_str} after {difference} ms')

called Mike after 13.4580078125 ms
called World after 105.0361328125 ms
called There after 150.573974609375 ms


In [ ]:
# Calculate ms offset from start time

Extension - Make it run on separate thread

# Test Cases

In [ ]:
#Test by checking the time each method was called,and the time it was supposed to be called

# Retrospective